# "Living Audio Irish"
> "TTS test corpus for Irish from IDLAK"

- toc: false
- branch: master
- comments: true
- categories: [speech, dataset]

In [ ]:
%%capture
!wget https://ia800700.us.archive.org/6/items/ga.ie.cll.48000.tar/ga.ie.cll.48000.tar.gz

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/Idlak/Living-Audio-Dataset/master/ga/text.xml

In [ ]:
%%capture
!tar zxvf ga.ie.cll.48000.tar.gz
!rm ga.ie.cll.48000.tar.gz

In [6]:
%%capture
!pip install bs4

     |████████████████████████████████| 115 kB 1.2 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=63c582622b354502078cdd26134b94c00db144c09bb108d80d559383ae58c331
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [9]:
from bs4 import BeautifulSoup
import unicodedata
soup = BeautifulSoup(open('text.xml').read(), 'lxml')
dataset = list()
for entry in soup.find_all('fileid'):
    current = dict()
    current['id'] = entry['id']
    current['text'] = unicodedata.normalize('NFC', entry.text.strip())
    dataset.append(current)

In [ ]:
!rm text.xml

In [10]:
def is_upper_vowel(letter):
  if letter in ['A', 'E', 'I', 'O', 'U', 'Á', 'É', 'Í', 'Ó', 'Ú']:
    return True
  else:
    return False

def irish_lower(word):
  if len(word) > 1 and word[0] in ['n', 't'] and is_upper_vowel(word[1]):
    return word[0] + '-' + word[1:].lower()
  else:
    return word.lower()

def irish_lower_sentence(sentence):
  return " ".join([irish_lower(w) for w in sentence.split(" ")])

In [11]:
import re
hyphens = 'cll_z0001_713 cll_z0001_804 cll_z0002_069 cll_z0002_296 cll_z0002_448 cll_z0002_481 cll_z0002_484 cll_z0002_495'.split(' ')
for entry in dataset:
    tmp = entry['text']
    tmp = re.sub(' \- ', ' ', tmp)
    tmp = re.sub(' – ', ' ', tmp)
    tmp = re.sub('[‘“”\"\.\?!,–—;:]', '', tmp)
    if entry['id'] in hyphens:
        tmp = re.sub('\'', '', tmp)
    entry['sentence'] = irish_lower_sentence(tmp)

In [12]:
for entry in dataset:
    entry['speaker'] = 'caoimhin'
    entry['accent'] = 'dublin'
    entry['gender'] = 'male'
    entry['path'] = '../input/living-audio-irish-speech-corpus/48000_orig/{}.wav'.format(entry['id'])

In [14]:
import json
datasetjson = json.dumps(dataset)
jsonf = open("living-audio.json", "w")
jsonf.write(datasetjson)
jsonf.close()

In [22]:
!wget https://raw.githubusercontent.com/Idlak/idlak/master/idlak-data/ga/ie/lexicon-default.xml

--2021-04-20 21:54:40--  https://raw.githubusercontent.com/Idlak/idlak/master/idlak-data/ga/ie/lexicon-default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405337 (396K) [text/plain]
Saving to: ‘lexicon-default.xml’

lexicon-default.xml 100%[===================>] 395.84K  --.-KB/s    in 0.03s   

2021-04-20 21:54:40 (14.8 MB/s) - ‘lexicon-default.xml’ saved [405337/405337]



In [23]:
from bs4 import BeautifulSoup
import unicodedata
soup = BeautifulSoup(open('lexicon-default.xml').read(), 'lxml')
lexicon = []
for entry in soup.find_all('lex'):
    current = {}
    current['pron'] = entry['pron']
    current['text'] = unicodedata.normalize('NFC', entry.text.strip())
    lexicon.append(current)

In [24]:
lexiconjson = json.dumps(lexicon)
jsonf = open("ga-lexicon.json", "w")
jsonf.write(lexiconjson)
jsonf.close()

In [ ]:
!rm lexicon-default.xml

In [26]:
# MFA/Kaldi style lexicon
with open('lexicon.txt', 'w') as lextxt:
    for lex in lexicon:
        text = lex['text']
        cleaned = lex['pron'].replace('0', '').replace('1', '').replace('2', '')
        lextxt.write(f'{text} {cleaned}\n')